In [8]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd

# Função para capturar links dos campeonatos
def capturar_links(url, card_class):
    response = requests.get(url)
    response.raise_for_status()  # Verificar se a requisição foi bem-sucedida

    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar o <div> com a classe 'cards'
    cards_div = soup.find('div', class_='cards')

    # Verificar se o <div> foi encontrado
    if not cards_div:
        print("Elemento <div> com a classe 'cards' não encontrado.")
        return []

    # Encontrar o <div> com a classe especificada dentro do <div> com a classe 'cards'
    card_div = cards_div.find('div', class_=card_class)

    # Verificar se o <div> foi encontrado
    if not card_div:
        print(f"Elemento <div> com a classe '{card_class}' não encontrado.")
        return []

    # Encontrar o <ul> com a classe 'card__list is--hidden' dentro do <div>
    card_list_ul = card_div.find('ul', class_='card__list is--hidden')

    # Verificar se o <ul> foi encontrado
    if not card_list_ul:
        print("Elemento <ul> com a classe 'card__list is--hidden' não encontrado.")
        return []

    # Encontrar todos os elementos <li> com a classe 'card__list-item'
    list_items = card_list_ul.find_all('li', class_='card__list-item')

    # Inicializar a lista para armazenar os links
    links = []

    # Iterar sobre os itens da lista e pegar os links
    for list_item in list_items:
        link_tag = list_item.find('a')
        if link_tag and 'href' in link_tag.attrs:
            link_href = link_tag['href']
            links.append(link_href)

    return links

# Função para capturar os links das equipes
def capturar_links_equipes(campeonatos):
    equipes = {}

    for jogo, links in campeonatos.items():
        equipes[jogo] = []
        for link in links:
            try:
                response = requests.get("https://liquipedia.net" + link)
                response.raise_for_status()
            except requests.exceptions.HTTPError as http_err:
                print(f"HTTP error occurred for {link}: {http_err}")
                continue
            except Exception as err:
                print(f"Other error occurred for {link}: {err}")
                continue

            soup = BeautifulSoup(response.content, 'html.parser')

            if jogo == "Dota 2":
                # Encontrar o <div> com style "max-width:1000px;"
                max_width_div = soup.find('div', style="max-width:1000px;")

                if max_width_div:
                    # Encontrar todos os <div> com a classe "template-box" dentro do <div> com style "max-width:1000px;"
                    template_box_divs = max_width_div.find_all('div', class_='template-box')

                    for template_box in template_box_divs:
                        center_tag = template_box.find('center', style="font-weight:bold")
                        if center_tag:
                            link_tag = center_tag.find('a')
                            if link_tag and 'href' in link_tag.attrs:
                                equipes[jogo].append(link_tag['href'])
            elif jogo == "Valorant":
                # Encontrar todos os <div> com a classe "template-box"
                template_box_divs = soup.find_all('div', class_='template-box')

                for template_box in template_box_divs:
                    center_tag = template_box.find('center', style="font-weight:bold")
                    if center_tag:
                        link_tag = center_tag.find('a')
                        if link_tag and 'href' in link_tag.attrs:
                            equipes[jogo].append(link_tag['href'])

    return equipes

# Função para capturar jogadores e suas informações
def capturar_jogadores(equipes):
    jogadores = []

    for jogo, links in equipes.items():
        for link in links:
            full_url = "https://liquipedia.net" + link
            try:
                response = requests.get(full_url)
                response.raise_for_status()
            except requests.exceptions.HTTPError as http_err:
                print(f"HTTP error occurred for {full_url}: {http_err}")
                continue
            except Exception as err:
                print(f"Other error occurred for {full_url}: {err}")
                continue

            # Parsear o conteúdo da página com lxml para facilitar a busca por xpath
            tree = html.fromstring(response.content)

            # Determinar o xpath correto baseado no jogo
            equipe_xpath = "//span[@class='mw-page-title-main']"

            # Capturar o nome da equipe usando o xpath
            equipe_elemento = tree.xpath(equipe_xpath)
            if equipe_elemento:
                equipe_nome = equipe_elemento[0].text.strip()  # Extraindo o texto do elemento
            else:
                equipe_nome = "Equipe Desconhecida"
            
            soup = BeautifulSoup(response.content, 'html.parser')

            table_responsive_div = soup.find('div', class_='table-responsive')

            if table_responsive_div:
                player_rows = table_responsive_div.find_all('tr', class_='Player')

                for player_row in player_rows:
                    name_td = player_row.find('td', class_='Name')
                    if name_td:
                        mobile_stuff_div = name_td.find('div', class_='MobileStuff')
                        if mobile_stuff_div:
                            jogador_nome = mobile_stuff_div.get_text(strip=True).strip('()').strip()
                            if jogador_nome:  # Verificar se o nome não está vazio
                                jogadores.append((jogador_nome, jogo, equipe_nome))

    return jogadores

# URL do site a ser acessado
url = 'https://liquipedia.net/'

# Especificar a classe dos cards desejados
card_classes = {
    'Dota 2': 'card dota2-card',
    'Valorant': 'card valorant-card'
}

# Capturar os links dos campeonatos
campeonatos = {}
for jogo, card_class in card_classes.items():
    campeonatos[jogo] = capturar_links(url, card_class)

# Capturar os links das equipes
equipes = capturar_links_equipes(campeonatos)

# Capturar os jogadores e suas informações (sem limitação)
jogadores = capturar_jogadores(equipes)

# Criar DataFrame
df = pd.DataFrame(jogadores, columns=['atletas', 'Modalidade', 'equipes'])

# Salvando o DataFrame em um arquivo CSV
df.to_csv('jogadores_esports.csv', index=False)

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL base do site Liquipedia
base_url = 'https://liquipedia.net'

# URL da página inicial do site Liquipedia
url = 'https://liquipedia.net/'

# Enviando uma requisição GET para a página inicial
response = requests.get(url)

# Listas para armazenar os dados
equipes_dota2 = []
atletas_dota2 = []
nome_equipe_dota2 = []
nome_campeonatos_dota2 = []
modalidade = []

if response.status_code == 200:
    # Parsing do conteúdo da página com BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrando todos os elementos <a> com a classe 'card__list-link'
    elementos_a = soup.find_all('a', class_='card__list-link')
    
    # Extraindo os links (href) e anexando na lista 'campeonatos'
    campeonatos = [elemento.get('href') for elemento in elementos_a]
    
    # Criando a nova lista 'campeonatos_dota2' com os links que contêm 'dota2'
    campeonatos_dota2 = [base_url + link for link in campeonatos if 'dota2' in link]
    
    # Iterando pelos links de 'campeonatos_dota2'
    for campeonato_link in campeonatos_dota2:
        response_camp = requests.get(campeonato_link)
        
        if response_camp.status_code == 200:
            soup_camp = BeautifulSoup(response_camp.content, 'html.parser')
            
            # Capturando o nome do campeonato diretamente da página do campeonato
            nome_campeonato_element = soup_camp.find('h1', class_='firstHeading')
            if nome_campeonato_element:
                nome_campeonato = nome_campeonato_element.get_text(strip=True)
            else:
                nome_campeonato = "Nome do Campeonato Não Encontrado"
            
            # Encontrando os elementos <a> dentro de <div> com a classe 'teamcard'
            div_teamcards = soup_camp.find_all('div', class_='teamcard')
            
            for div in div_teamcards:
                # Dentro de cada 'teamcard', buscamos os links das equipes
                equipe_link = div.find('a', href=True)
                if equipe_link:
                    equipe_href = equipe_link.get('href')
                    if equipe_href:
                        equipe_url = base_url + equipe_href
                        equipes_dota2.append(equipe_url)
                        
                        # Acessando a página da equipe para capturar os nomes dos atletas
                        response_equipe = requests.get(equipe_url)
                        if response_equipe.status_code == 200:
                            soup_equipe = BeautifulSoup(response_equipe.content, 'html.parser')
                            
                            # Capturando o nome da equipe
                            nome_equipe = soup_equipe.find('span', dir='auto').get_text(strip=True)
                            
                            nomes_achados = False
                            # Encontrando os elementos <div> com a classe 'LargeStuff'
                            atletas = soup_equipe.find_all('div', class_='LargeStuff')
                            for atleta in atletas:
                                nome_atleta = atleta.get_text(strip=True)
                                if nome_atleta:
                                    atletas_dota2.append(nome_atleta)
                                    nome_equipe_dota2.append(nome_equipe)
                                    nome_campeonatos_dota2.append(nome_campeonato)  # Adiciona o nome do campeonato
                                    modalidade.append('Dota 2')  # Adiciona a modalidade
                                    nomes_achados = True
                            
                            if nomes_achados:
                                print(f'Nomes capturados com sucesso na página {equipe_url}')
                            else:
                                print(f'Não foi possível capturar nomes na página {equipe_url}')
                        else:
                            print(f'Erro ao acessar a página da equipe: {equipe_url}')
        
        else:
            print(f'Erro ao acessar {campeonato_link}: {response_camp.status_code}')
    
    # Removendo duplicatas da lista 'equipes_dota2'
    equipes_dota2 = list(set(equipes_dota2))
    
    # Criando um DataFrame com os resultados
    df_atletas = pd.DataFrame({
        'Nome Atleta': atletas_dota2,
        'Nome Equipe': nome_equipe_dota2,
        'Nome Campeonato': nome_campeonatos_dota2,
        'Modalidade': modalidade
    })
    
    # Salvando o DataFrame em um arquivo CSV
    df_atletas.to_csv('atletas_dota2.csv', index=False)

    print(df_atletas)

else:
    print(f'Erro ao acessar a página: {response.status_code}')

Nomes capturados com sucesso na página https://liquipedia.net/dota2/Team_Spirit
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Xtreme_Gaming
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Team_Falcons
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Team_Liquid
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Gaimin_Gladiators
Nomes capturados com sucesso na página https://liquipedia.net/dota2/BetBoom_Team
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Cloud9
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Tundra_Esports
Nomes capturados com sucesso na página https://liquipedia.net/dota2/1w_Team
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Team_Zero
Nomes capturados com sucesso na página https://liquipedia.net/dota2/G2_x_iG
Nomes capturados com sucesso na página https://liquipedia.net/dota2/Talon_Esports
Nomes capturados com sucesso na página h

In [21]:
df_atletas.to_csv('atletas_dota2.csv', index=False)

In [23]:
print(campeonatos_dota2)

['https://liquipedia.net/dota2/The_International/2024', 'https://liquipedia.net/dota2/The_International/2024/Group_Stage', 'https://liquipedia.net/dota2/The_International', 'https://liquipedia.net/dota2/The_International/2024/Main_Event', 'https://liquipedia.net/dota2/The_International/2024/North_America']
